# DSC 80: Project 02

### Checkpoint Due Date: Thursday, April 22 11:59:59 PM (Q1-5)

### Final Due Date: Thursday, April 29 11:59:59 PM

---
# Instructions

This Jupyter Notebook contains the statements of the problems and provides code and markdown cells to display your answers to the problems.  
* Like the lab, your coding work will be developed in the accompanying `projectXX.py` file, that will be imported into the current notebook. This code will be autograded.

**Do not change the function names in the `*.py` file**
- The functions in the `*.py` file are how your assignment is graded, and they are graded by their name. The dictionary at the end of the file (`GRADED FUNCTIONS`) contains the "grading list". The final function in the file allows your doctests to check that all the necessary functions exist.
- If you changed something you weren't supposed to, just use git to revert!

**Tips for developing in the .py file**:
- Do not change the function names in the starter code; grading is done using these function names.
- Do not change the docstrings in the functions. These are there to tell you if your work is on the right track!
- You are encouraged to write your own additional functions to solve the HW! 
    - Developing in python usually consists of larger files, with many short functions.
    - You may write your other functions in an additional `.py` file that you import in `projectXX.py` (much like we do in the notebook).
- Always document your code!

## Checkpoint Instructions

* The checkpoint requires you to turn in **questions 1-5**; 
* **The checkpoint will be graded for *approximate* correctness: easier than the final tests; harder than the doctests.**


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import project02 as proj

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import os

# An Investigation into Flight Delays

### The flights dataset

The department of transportation has all flight delays for listed years on their [website](https://catalog.data.gov/dataset/airline-on-time-performance-and-causes-of-flight-delays-on-time-data). There are data for the years 1987 - 2018. See the description of columns in `data/columns.txt`.

This project will look at a single year (2015) to keep the analysis "simple", which is available at the URL below (*NOT* on the data.gov site).


* To download the flights dataset to your computer, use [this link](https://dsc80-fa19-data.s3-us-west-2.amazonaws.com/project02/flight-delays.zip), unzip the file, and place `flights.csv` in your project directory.

* To download the dataset on `datahub.ucsd.edu` (this works on your computer as well!):
    - Open the terminal in datahub ("new > Terminal")
    - Change the directory to where you want your data (e.g. `cd [ASSIGNMENT_PATH]/data`)
    - Download the unzipped dataset using these commands:
        1. `wget https://dsc80-fa19-data.s3-us-west-2.amazonaws.com/project02/flight-delays.zip`
        2. `unzip flight-delays.zip`
    - `flights.csv` should be in the directory.
    
**NOTE: The unzipped files must be in the `project02/data` directory in order for the doctests to work!**

### Creating your datasets

**Question 1**

The flights dataset for 2015 is not small (~600MB). While you could likely load the entire dataset into Pandas on your laptop, if you wanted to work with more than one year, this would quickly become difficult (the data is available for 1987-2018). Therefore, we will filter down the dataset into two smaller files without ever reading the larger dataset fully into memory. We are going to create two smaller datasets:

1. All flights arriving or departing from San Diego International Airport in 2015.
    - You can find a list of all the airport codes in the United States [here](https://www.leonardsguide.com/us-airport-codes.shtml).
2. All flights flown by either JetBlue or Southwest Airline in 2015.

---

To do this, you are going to use the `chunksize=N` keyword in Pandas `read_csv` to read the flights dataset in blocks of `N` lines. When you use this keyword argument, `pd.read_csv(fp, chunksize=N)` becomes a *iterator* that iterates through dataframes of length N until you have reached the end of the dataset. A typical pattern looks like:
```
L = pd.read_csv(filepath, chunksize=1000)
for df in L:
    process(df)
```
Where each `df` is a dataframe of length 1000. 

The processing you are going to do is:
1. Iterate through the dataset, chunk-by-chunk,
2. Filtering out rows of each chunk
3. Incrementally add to a filtered csv file (since the data is perhaps too big to keep in memory). Keep in mind, if you want to keep writing to the same file, the mode='a' keyword in the `.to_csv` method can be helpful when calling it in the loop (a stands for 'append')

---

Write two functions that create the datasets below, using the 'chunking' pattern described above. Your functions should use `chunksize` of 10000.
1. `get_san` which takes in a filepath containing all flights and a filepath where filtered dataset #1 is written. The function should return `None`.
1. `get_jb_sw` which takes in a filepath containing all flights and a filepath where filtered dataset #2 is written. The function should return `None`.

*Remark 1*: **Gradescope autograding servers are quite small and can't load this dataset into memory** -- so your code that reads in the large `flights.csv` dataset *must* work with chunks of the dataset one at a time to pass!

*Remark 2:* You can check your work using the datasets included in the zip file!

Remember to close your file properly!

In [40]:
infp = os.path.join('data', 'flights.csv')
outfp = os.path.join('data', 'jetblue_sw.csv')

In [41]:
L = pd.read_csv(infp, chunksize=1000)
df = next(L)
fil_df = df[(df['AIRLINE'] == 'B6') | (df['AIRLINE'] == 'WN')]
fil_df.to_csv(outfp, mode='a', index=False)
for df in L:
    fil_df = df[(df['AIRLINE'] == 'B6') | (df['AIRLINE'] == 'WN')]
    fil_df.to_csv(outfp, mode='a', index=False, header=False)

In [43]:
pd.read_csv(os.path.join('flight-delays', 'jetblue_or_sw.csv'))

E:\Python\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,B6,304,N607JB,SJU,JFK,155,...,501.0,11.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,B6,1990,N597JB,SJU,EWR,206,...,516.0,4.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,B6,668,N653JB,PSE,MCO,255,...,451.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,B6,1030,N239JB,BQN,MCO,307,...,520.0,20.0,0,0,NaN,20.0,0.0,0.0,0.0,0.0
4,2015,1,1,4,B6,262,N627JB,SJU,BOS,330,...,619.0,-16.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1528898,2015,12,31,4,B6,688,N657JB,LAX,BOS,2359,...,753.0,-26.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1528899,2015,12,31,4,B6,745,N828JB,JFK,PSE,2359,...,430.0,-16.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1528900,2015,12,31,4,B6,1503,N913JB,JFK,SJU,2359,...,432.0,-8.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1528901,2015,12,31,4,B6,333,N527JB,MCO,SJU,2359,...,330.0,-10.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
pd.read_csv(outfp)

E:\Python\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,B6,304,N607JB,SJU,JFK,155,...,501.0,11.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,B6,1990,N597JB,SJU,EWR,206,...,516.0,4.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,B6,668,N653JB,PSE,MCO,255,...,451.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,B6,1030,N239JB,BQN,MCO,307,...,520.0,20.0,0,0,NaN,20.0,0.0,0.0,0.0,0.0
4,2015,1,1,4,B6,262,N627JB,SJU,BOS,330,...,619.0,-16.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1528898,2015,12,31,4,B6,688,N657JB,LAX,BOS,2359,...,753.0,-26.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1528899,2015,12,31,4,B6,745,N828JB,JFK,PSE,2359,...,430.0,-16.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1528900,2015,12,31,4,B6,1503,N913JB,JFK,SJU,2359,...,432.0,-8.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1528901,2015,12,31,4,B6,333,N527JB,MCO,SJU,2359,...,330.0,-10.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


# Flight Delays to/from San Diego

The department of transportation has all flight delays for listed years on their [website](https://catalog.data.gov/dataset/airline-on-time-performance-and-causes-of-flight-delays-on-time-data). 

The zip file at the [URL](https://dsc80-fa19-data.s3-us-west-2.amazonaws.com/project02/flight-delays.zip) contains a file `to_from_san.csv` that consists of all flights either to or from SAN (San Diego) in 2015 -- i.e. the output of Question 1. This dataset should match the dataset that your code returned in question 1.

Read in `to_from_san.csv` using `read_csv` and inspect the dataframe for an initial assessment about the data quality.

In [44]:
# Run this cell
to_from_san_filepath = os.path.join('data', 'to_from_san.csv')
flights = pd.read_csv(to_from_san_filepath)

In [45]:
flights

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,DL,978,N693DL,SAN,SLC,615,...,906.0,-10.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,OO,5608,N930SW,SAN,LAX,615,...,702.0,-5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,WN,823,N7707C,SAN,BWI,620,...,1352.0,-23.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,WN,603,N461WN,SAN,MDW,620,...,1201.0,-29.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,UA,1192,N69804,SAN,DEN,620,...,936.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140409,2015,12,31,4,DL,1792,N1402A,SAN,ATL,2230,...,511.0,-13.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
140410,2015,12,31,4,UA,240,N30401,SAN,ORD,2235,...,405.0,-38.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
140411,2015,12,31,4,DL,1366,N341NW,SAN,DTW,2245,...,530.0,-34.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
140412,2015,12,31,4,UA,498,N37267,SFO,SAN,2252,...,6.0,-15.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


### Understanding the data types of the columns

**Question 2**:

* First, classify the *kind* of data each column in `flights` contains. Create a function `data_kinds` of zero variables which outputs a (hard-coded) dictionary of data kinds, keyed by column name, with values `Q`, `O`, `N` (for 'Quantitative', 'Ordinal', or 'Nominal').

* Second, decide the best data *type* for each column. Create a function `data_types` of zero variables which outputs a (hard-coded) dictionary of data types, keyed by column name, with values `str`, `int`, `float`, `bool`. 

*Remark 1*: A column which *should* be `int`s, but contains `NaN`, *must* be a float column. See Lecture 2 notes an explanation of `NaN` and data-types.

*Remark 2*: As with real data, some data processing decisions may be ambiguous here. Make the best decision using the information available to you. It may be helpful to (re)read the relevant [section of the textbook](https://afraenkel.github.io/practical-data-science/03/kinds-of-data.html). 
* Certain answers *may* have more than one correct answer (in these cases, more than one choice gets full credit),
* All answers will be graded for partial credit (some wrong answers are more wrong than other).
There are many columns, so don't worry about the correctness of any given one; do make sure you are thinking about what's contained in a column critically!

In [46]:
proj.data_kinds()

{'YEAR': 'O',
 'MONTH': 'O',
 'DAY': 'O',
 'DAY_OF_WEEK': 'O',
 'AIRLINE': 'N',
 'FLIGHT_NUMBER': 'N',
 'TAIL_NUMBER': 'N',
 'ORIGIN_AIRPORT': 'N',
 'DESTINATION_AIRPORT': 'N',
 'SCHEDULED_DEPARTURE': 'O',
 'DEPARTURE_TIME': 'Q',
 'DEPARTURE_DELAY': 'Q',
 'TAXI_OUT': 'Q',
 'WHEELS_OFF': 'Q',
 'SCHEDULED_TIME': 'O',
 'ELAPSED_TIME': 'Q',
 'AIR_TIME': 'Q',
 'DISTANCE': 'Q',
 'WHEELS_ON': 'Q',
 'TAXI_IN': 'Q',
 'SCHEDULED_ARRIVAL': 'O',
 'ARRIVAL_TIME': 'O',
 'ARRIVAL_DELAY': 'Q',
 'DIVERTED': 'N',
 'CANCELLED': 'N',
 'CANCELLATION_REASON': 'N',
 'AIR_SYSTEM_DELAY': 'Q',
 'SECURITY_DELAY': 'Q',
 'AIRLINE_DELAY': 'Q',
 'LATE_AIRCRAFT_DELAY': 'Q',
 'WEATHER_DELAY': 'Q'}

In [47]:
proj.data_types()

{'YEAR': 'int',
 'MONTH': 'int',
 'DAY': 'int',
 'DAY_OF_WEEK': 'int',
 'AIRLINE': 'str',
 'FLIGHT_NUMBER': 'int',
 'TAIL_NUMBER': 'str',
 'ORIGIN_AIRPORT': 'str',
 'DESTINATION_AIRPORT': 'str',
 'SCHEDULED_DEPARTURE': 'int',
 'DEPARTURE_TIME': 'float',
 'DEPARTURE_DELAY': 'float',
 'TAXI_OUT': 'float',
 'WHEELS_OFF': 'float',
 'SCHEDULED_TIME': 'int',
 'ELAPSED_TIME': 'float',
 'AIR_TIME': 'float',
 'DISTANCE': 'int',
 'WHEELS_ON': 'float',
 'TAXI_IN': 'float',
 'SCHEDULED_ARRIVAL': 'int',
 'ARRIVAL_TIME': 'float',
 'ARRIVAL_DELAY': 'float',
 'DIVERTED': 'bool',
 'CANCELLED': 'bool',
 'CANCELLATION_REASON': 'str',
 'AIR_SYSTEM_DELAY': 'float',
 'SECURITY_DELAY': 'float',
 'AIRLINE_DELAY': 'float',
 'LATE_AIRCRAFT_DELAY': 'float',
 'WEATHER_DELAY': 'float'}

### Read in the typed flights data

Read in the flights data using your dictionary of data-types in `read_csv`. This both speeds up parsing, as well as gives you the correct data-types upon reading (which columns would pandas *parse incorrectly* if you didn't use a `dtype` dictionary?)

In [48]:
# Run this cell
dtypes = proj.data_types()
flights = pd.read_csv(to_from_san_filepath, dtype=dtypes)

In [49]:
flights

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,DL,978,N693DL,SAN,SLC,615,...,906.0,-10.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,OO,5608,N930SW,SAN,LAX,615,...,702.0,-5.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,WN,823,N7707C,SAN,BWI,620,...,1352.0,-23.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,WN,603,N461WN,SAN,MDW,620,...,1201.0,-29.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,UA,1192,N69804,SAN,DEN,620,...,936.0,-9.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140409,2015,12,31,4,DL,1792,N1402A,SAN,ATL,2230,...,511.0,-13.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
140410,2015,12,31,4,UA,240,N30401,SAN,ORD,2235,...,405.0,-38.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
140411,2015,12,31,4,DL,1366,N341NW,SAN,DTW,2245,...,530.0,-34.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
140412,2015,12,31,4,UA,498,N37267,SFO,SAN,2252,...,6.0,-15.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN


**Question 3 (Basic Stats):**

Define a function `basic_stats` that takes a dataframe `flights` and outputs a dataframe that contains statistics for flights arriving/departing for SAN. That is, the output should have two rows, indexed by `ARRIVING` and `DEPARTING`, and have the following columns:

1. The number of arriving/departing flights to/from SAN (`count`).
    - If a flight scheduled to arrive at SAN never arrives, it still counts as an arriving flight.
2. The mean flight (arrival) delay of arriving/departing flights to/from SAN (`mean_delay`).
3. The median flight (arrival) delay of arriving/departing flights to/from SAN (`median_delay`).
4. The airline code of the airline with the single longest flight (arrival) delay among all flights arriving/departing to/from SAN (`airline`).
5. A list of the three months with the greatest number of arriving/departing flights to/from SAN, sorted from greatest to least (`top_months`).

*Remark:* Null values should not be considered when computing statistics; however, think about whether e.g. the average flight delay is likely higher or lower that the "true mean" by making this choice.

*Hint*: Use `groupbby` and the fact that `aggregate` can take in a dictionary as an argument.

In [50]:
proj.basic_stats(flights)

,count,mean_delay,median_delay,airline,top_months
ARRIVING,70207,3.676826,-4.0,AA,"[7, 8, 6]"
DEPARTING,70207,3.328988,-5.0,AA,"[7, 8, 6]"


### Understanding flight delays: Departures, Arrivals, and everything in-between

**Question 4**

Often `DEPARTURE_DELAY` is thought to be the main cause of a flight delay -- i.e., when the flight is late pushing off from the gate. 

However, there are other ways that flights can be late: waiting on the tarmac, headwinds, turbulence, circling a busy airport, and waiting for a gate after landing. First, we will analyze all the ways in which a flight can be delayed.

* First, create a function `depart_arrive_stats` that takes in a dataframe like `flights` and calculates the following quantities in a series, indexed by `late1`, `late2`, `late3`:
    - `late1`: the proportion of flights from/to SAN that leave late, but arrive early or on-time.
    - `late2`: the proportion of flights from/to SAN that leave early, or on-time, but arrive late.
    - `late3`: the proportion of flights from/to SAN that both left late and arrived late.
    
* Second, create a function `depart_arrive_stats_by_month` that takes in a dataframe like `flights` and calculates the quantities above broken down by *month*. That is, the output is a dataframe, indexed by `MONTH`, with columns given by `late1`, `late2`, `late3`.

*Remark 1:* Does this question reveal any data quality issues? Can you pinpoint when these issues occur?

*Remark 2:* A flight is considered late if it departed/arrived any time later than its planned departure/arrival time.

*Remark 3:* You do not need to manually calculate the delay time for those delays that are `NaN`.

In [73]:
late1 = ((flights['DEPARTURE_DELAY'] > 0) & (flights['ARRIVAL_DELAY'] < 0)).mean()
late2 = ((flights['DEPARTURE_DELAY'] <= 0) & (flights['ARRIVAL_DELAY'] > 0)).mean()
late3 = ((flights['DEPARTURE_DELAY'] > 0) & (flights['ARRIVAL_DELAY'] > 0)).mean()
pd.Series({'late1': late1, 'late2': late2, 'late3': late3})

late1    0.110188
late2    0.089329
late3    0.278804
dtype: float64

In [77]:
proj.depart_arrive_stats(flights)

late1    0.110188
late2    0.089329
late3    0.278804
dtype: float64

In [76]:
proj.depart_arrive_stats_by_month(flights)

,late1,late2,late3
MONTH,,,
1,0.109696,0.095645,0.270501
2,0.109783,0.096915,0.289925
3,0.118683,0.089240,0.258309
4,0.108066,0.095064,0.264326
5,0.104960,0.083498,0.285151
6,0.110421,0.088248,0.330968
7,0.105271,0.095455,0.335799
8,0.114666,0.086890,0.272450
9,0.102066,0.087890,0.185905


### Flight delays and day of the week

**Question 5**

Next, we'd like to understand the flight traffic to/from SAN by day of the week. Day of the week is specified by integers 1 through 7; verify for yourself which integer corresponds to which day (hint: you have the *date* for each flight as well!).

Next create two functions to understand both the amount of traffic and the average flight delay of flights for each airline by day-of-the week. We both want to understand *presence* each airline has as well as their *performance*.

1. Create a function `cnts_by_airline_dow` that takes in a dataframe like `flights` and outputs a dataframe that answers the following question: Given any `AIRLINE` and `DAY_OF_WEEK`, how many flights were there (in 2015)?


2. Create a function `mean_by_airline_dow` that takes in a dataframe like `flights` and outputs a dataframe that answers the following question: Given any `AIRLINE` and `DAY_OF_WEEK`, what is the average `ARRIVAL_DELAY` (in 2015)?

Both dataframes should have a column for each distinct value of `AIRLINE` and a row for each `DAY_OF_WEEK`.

*Hint:* `pivot_table` should be useful here!

Your output should have the *form* of the table below (not the entries themselves!)

<img src="data/pivot.png" />

In [79]:
counts = flights.groupby(['DAY_OF_WEEK', 'AIRLINE'], as_index=False)['FLIGHT_NUMBER'].count()
counts.pivot(index='DAY_OF_WEEK', columns='AIRLINE', values='FLIGHT_NUMBER')

AIRLINE,AA,AS,B6,DL,F9,HA,NK,OO,UA,US,VX,WN
DAY_OF_WEEK,,,,,,,,,,,,
1,1915,1564,384,1753,218,96,531,1859,2369,495,536,9238
2,1872,1345,359,1647,210,96,529,1829,2211,463,544,9300
3,1922,1428,359,1719,227,96,530,1814,2293,472,542,9328
4,1888,1551,380,1729,220,96,531,1835,2328,510,531,9108
5,1859,1449,376,1701,208,94,518,1813,2279,505,523,9006
6,1755,1476,270,1433,195,94,518,1837,1743,496,365,7201
7,1886,1465,387,1726,214,96,532,1889,2172,503,443,8587


### Understanding null values in the flights data

**Question 6 (Missing by Design)**

Now we would like to understand how data is missing in the flights data. First, compute the proportion of each column of `flights` which are non-null. Do not turn this in, but it will be useful information in doing the next few problems.

Recall that a column is *missing by design* if you can determine when the entry of a column is missing based solely on other data in the same row. That is
* there is *no randomness* in determining when an entry is missing.
* you can describe when the column is missing a value with a logical (not random) condition.
* you can express which rows will have missing values in terms of logical statements about the *other* columns in the same row.

For this question, verify the following columns are *missing by design*:
* The column `ARRIVAL_DELAY` is *missing by design*. Create a function `predict_null_arrival_delay` that doesn't depend on the values of `ARRIVAL_DELAY`, that:
    - Takes in a row of the flights data (that is, a Series)
    - Returns `True` if and only if the `ARRIVAL_DELAY` is null; otherwise it returns `False`.
    - Since the function doesn't depend on `ARRIVAL_DELAY`, it should work on a row even if the `ARRIVAL_DELAY` index is dropped.
    - You can check your function by using `flights.drop('ARRIVAL_DELAY', axis=1).apply(predict_null, axis=1)` and compare it to the `ARRIVAL_DELAY` column!


* The column `AIRLINE_DELAY` is *missing by design*. As above, create a function `predict_null_airline_delay` that doesn't depend on the values of `AIRLINE_DELAY`, that:
    - Takes in a row of the flights data (that is, a Series)
    - Returns `True` if and only if the `AIRLINE_DELAY` is null; otherwise it returns `False`.


**Question 7 (Missingness Types)**

Now we'd like to determine missingness of the column `DEPARTURE_DELAY`. In particular, we'd like to perform a permutation test to determine the missingness of `DEPARTURE_DELAY` dependent on the column `AIRLINE`.

* Create a function `perm4missing`:
    - that takes in `flights`, a column `col`, and a number `N` and 
    - returns the p-value of the test (using `N` simulations) that determines if `DEPARTURE_DELAY` is MAR dependent on `col`. That is `perm4missing(flights, 'AIRLINE', N)` should return the p-value for the test above.
    - *Remark*: to help your work, create helper functions whose output you can plot, to assess the correctness of your p-value!
    
* Use your function above to determine the columns `col` for which "`DEPARTURE_DELAY` is MAR dependent on `col`" using a significance level of 0.01. Only consider the categorical columns `YEAR`,`DAY_OF_WEEK`, `AIRLINE`,`DIVERTED`, `CANCELLATION_REASON`. Return your answer in a (hard-coded) list returned by a function called `dependent_cols`.

* Create a function `missing_types` of zero variables, which:
    - Returns a Series, indexed by the following columns of `flights`: `CANCELLED`, `CANCELLATION_REASON`, `TAIL_NUMBER`, `ARRIVAL_TIME`.
    - The values should contain the most-likely missingness type of each column. 
    - The values of this Series should be `MD, MCAR, MAR, MNAR, NaN` (use `NaN` if there are no missing values). 


*Hint:* for missingness permutation tests, try using TVD as a test statistic! You are taking the TVD between the distributions of `True` and `False` for those non-numerical columns; `True` being the proportions of null values, `False` being the proportions of non-null values. Check the lecture slides for more information.

# Simpson's Paradox: JetBlue vs Southwest

The remainder of the questions investigates the presence of Simpson's paradox in the flights dataset. Read through the final slides of lecture 05, as well as [the book](https://afraenkel.github.io/practical-data-science/05/understanding-aggregations.html#simpsons-paradox) for a summary of Simpson's Paradox and related links.

The csv file `southwest_vs_jetblue.csv` contains all Southwest and JetBlue flights in 2015.

In this dataset, we are going to verify the following occurrences of Simpson's Paradox: For a given set of airports,
* The average departure delay of Southwest is greater than (or less than) the average departure delay of JetBlue.
* Airport by airport, the average departure delay of Southwest is *less* than (or greater than) the average departure delay of JetBlue.

That is, the inequalities of the average flight delays between the two airlines are reversed when viewed at the level of each airport. In fact this reversal holds for *every* airport being considered.

In [ ]:
jb_sw_filepath = os.path.join('data', 'jetblue_or_sw.csv')
dtype = proj.data_types()

# The `usecols` keyword:
# choose *only* the columns you need to reduce your memory footprint!
usecols = ...

jb_sw = pd.read_csv(jb_sw_filepath, dtype=dtype, usecols=usecols)

**Question 8**

Filter the dataset `jb_sw` to flights *originating* from the following 10 airports: ABQ, BDL, BUR, DCA, MSY, PBI, PHX, RNO, SJC, SLC.

Illustrate Simpson's paradox with this table:
* Calculate the proportion of each airline's flights that are delayed (at each of the 10 airports):
    - Create a function `prop_delayed_by_airline` that takes in a dataframe like `jb_sw` and returns a DataFrame indexed by airline that contains the proportion of each airline's flights that are delayed.
* Calculate these proportions across all airports in the dataset (at each of the 10 airports):
    - Create a function `prop_delayed_by_airline_airport` that takes in a dataframe like `jb_sw` and returns a DataFrame, with columns given by airports, indexed by airline, that contains the proportion of each airline's flights that are delayed at each airport.

*Remark 1:* For the purpose of this question, a canceled flight is **not** considered delayed.

*Remark 2:* Make sure that the functions only work with the columns that are absolutely necessary for the question to avoid out of memory errors!

Verify that Simpson's paradox is present in this output! 

Can you explain *why* Simpson's paradox is occurring? (Hint: where are these airports located? Which have the most flights?)

**Question 9**

Your work above illustrates Simpson's paradox on the specific dataset of flights originating from 10 specific airports. However, this still requires you to look at two dataframe to see if the paradox is present. Now, you will create a function that verifies Simpson's paradox in general. You will do this by writing code to compare the two dataframes, instead of inspecting them manually.

Create a function `verify_simpson` that returns a boolean output regarding if the paradox is present.
```
verify_simpson(df, group1, group2, occur)
```
- df is a dataframe (e.g. jb_sw),
- group1 is the first group being aggregated against (e.g. `AIRLINE`),
- group2 is the second group being aggregated against (e.g. `ORIGIN_AIRPORT`),
- occur is a column with values {0, 1}, denoting if an event occurred for that individual.
  (e.g. "1 if flight was delayed" and "0 if flight was not delayed")

`verify_simpson` should return `True` only if there is a reversal for *every* value of `group2` (e.g. for every airport).

Example:

Consider the following dataframe `df` with columns `treatment`, `stone_size`, and `success`:

|treatment|stone_size|success|
|---|---|---|
|A|small|1|
|B|small|1|
|...|...|...|
|A|large|0|
|B|small|0|
|B|small|1|

`df` corresponds to the following diagram:
<img src="https://miro.medium.com/max/996/1*IfVjfdGD7RPwLDC6WzT9Uw.png" style="width: 300px"/>

Here, `verify_simpson(df, 'treatment', 'stone_size', 'success')` should return `True`.

Verify that you function works on the previous question!

### Bonus problem (worth zero points)

This question is for fun and explores a very data-science type problem: can we automate finding examples of Simpson's Paradox? This is an active area of research (see for example: https://arxiv.org/pdf/1801.04385.pdf), but is a very accessible problem. While totally optional, this question can lead to pretty interesting self-driven projects!

**Question 10 (Searching for Simpson's Paradox):**

As you observed from the reading in the lecture notes, Simpson's Paradox often occurs due to some confounding factor among the columns of a dataset. In the case of gender balance in student admissions to academic departments at UC Berkeley, the confounding factor was the admission rate (i.e. how hard it is to gain admission to a department).

What might be a confounding factor be for flight delays among airports in question 8? Now you are going to write code to discover instances of Simpson's Paradox; that is, you will programmatically find interesting relationships present in the data.

Given the dataset `jb_sw`, we'd like to find new groups of airports, as in question 8, for which the statistics of flight delays between JetBlue and Southwest satisfy Simpson's Paradox.

Create a function `search_simpsons` that takes in the `jb_sw` dataset and a number `N`, and returns a list of `N` airports for which the proportion of flight delays between JetBlue and Southwest satisfies Simpson's Paradox.
- Only consider airports that have '3 letter codes',
- Only consider airports that have at least one JetBlue *and* Southwest flight.

*Remark 1:* Iterate through groups of airports of size `N` using `itertools.combinations` until you find a group that works. Make sure your function finishes, even if it doesn't find something.

*Remark 2:* You should be using your work from Question 9!

# Congratulations, you finished the project!

### Before you submit:
* Be sure you run the doctests on all your code in project02.py

### To submit:
* **Upload the .py file to gradescope**